<img align="left" src = https://linea.org.br/wp-content/themes/LIneA/imagens/logo-header.jpg width=100 style="padding: 20px"> 

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=160 style="padding: 20px">  

# Executing Rail with DES Survey data - FALTA ESCREVER NELE, a execução foi

**Contact author**: Heloisa da Silva Mengisztki ([heloisasmengisztki@gmail.com](mailto:heloisasmengisztki@gmail.com)) 

**Last verified run**: 2022-12-01 (YYYY-MM-DD) <br><br>

Here we are going to execute rail using the same sample used in FASE-1, the main reason is to see if we can obtain a familiar result using rail-bpz

In [ ]:
import os
import numpy as np
import pandas as pd
import qp
import tables_io
import matplotlib.pyplot as plt

import rail
from rail.core.utils import RAILDIR
from rail.core.data import TableHandle, PqHandle, ModelHandle, QPHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

from rail.estimation.algos.bpz_lite import Inform_BPZ_lite, BPZ_lite
from rail.evaluation.evaluator import Evaluator

from rail.estimation.algos.knnpz import Inform_KNearNeighPDF

In [ ]:
CURR_DIR = os.getcwd()
CURR_DIR, RAILDIR

DS = RailStage.data_store
DS.__class__.allow_overwrite = True
DS

### Reading some sample

In [ ]:
headers=[
    "u_auto", "u_auto_err", "u_det", "u_det_err",
    "g_auto", "g_auto_err", "g_det", "g_det_err", 
    "r_auto", "r_auto_err", "r_det", "r_det_err", 
    "i_auto", "i_auto_err", "i_det", "i_det_err", 
    "z_auto", "z_auto_err", "z_det", "z_det_err", 
    "Y_auto", "Y_auto_err", "Y_det", "Y_det_err", 
    "z", "ra", "dec", "z_flags", "field_id", "number"
]

test_sample = pd.read_csv("/home/heloisamengisztki/DATA/des_sv_main_standard.valid", delimiter=" ", names=headers)
train_sample = pd.read_csv("/home/heloisamengisztki/DATA/des_sv_main_standard.train", delimiter=" ", names=headers)

#### Spliting into train and test data

In [ ]:
train_sample.head()

In [ ]:
test_sample.head()

---

In [ ]:
columns_map = {
"field_id": "id",
"ra": "coord_ra",
"dec": "coord_dec",
"g_auto": "mag_g_lsst",
"g_auto_err": "mag_err_g_lsst",
"r_auto": "mag_r_lsst",
"r_auto_err": "mag_err_r_lsst",
"i_auto": "mag_i_lsst",
"i_auto_err": "mag_err_i_lsst",
"u_auto": "mag_u_lsst",
"u_auto_err": "mag_err_u_lsst",
"Y_auto": "mag_y_lsst",
"Y_auto_err": "mag_err_y_lsst",
"z_auto": "mag_z_lsst",
"z_auto_err": "mag_err_z_lsst",
"z": "redshift"
}

col_remapper_train = ColumnMapper.make_stage(name='col_remapper_train', columns=columns_map)
col_remapper_test = ColumnMapper.make_stage(name='col_remapper_test', columns=columns_map)

In [ ]:
train_sample = col_remapper_train(train_sample)

In [ ]:
col_remapper_train.get_data("output")

In [ ]:
bpz_columns_file = os.path.join(CURR_DIR, '../configs/bpz.columns')
bpz_columns_file

In [ ]:
bpz_columns_file = os.path.join(CURR_DIR, '../configs/bpz.columns')

inform_bpz = Inform_BPZ_lite.make_stage(
    name='inform_bpzlite', 
    model='trained_BPZ_output.pkl', 
    hdf5_groupname='', 
    columns_file=bpz_columns_file,
    prior_band="mag_i_lsst",
)
inform_bpz.config.to_dict()

Running Inform to compute the best fit prior parameters. What is fo and kt?

In [ ]:
inform_bpz.inform(train_sample)

___

In [ ]:
test_sample = col_remapper_test(test_sample)

In [ ]:
col_remapper_test.get_data("output")

In [ ]:
table_conv = TableConverter.make_stage(name='table_conv_test', output_format='numpyDict');
test_sample_table = table_conv(test_sample)

In [ ]:
estimate_bpz = BPZ_lite.make_stage(
    name='estimate_bpz', 
    hdf5_groupname='', 
    columns_file=bpz_columns_file, 
    #input="inprogress_output_table_conv_train.hdf5", 
    model=inform_bpz.get_handle('model'),
    filters="path",
    zmax = 3,
    chunk_size=20
)
estimate_bpz.config.to_dict()

In [ ]:
output_bpz = estimate_bpz.estimate(test_sample_table)

In [ ]:
test_data = col_remapper_test.get_data("output")
test_data

In [ ]:
evaluator = Evaluator.make_stage(name=f'bpz_eval', truth=test_data)
#evaluator.config.to_dict()
result_dict = evaluator.evaluate(output_bpz, test_data)

____

In [ ]:
results_tables = tables_io.convertObj(DS.read("output_estimate_bpz").build_tables()['ancil'], tables_io.types.PD_DATAFRAME)
zmode = results_tables['zmode']
tables_io.convertObj(result_dict.data, tables_io.types.PD_DATAFRAME)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data['redshift'],zmode,s=1,c='k',label='simple bpz mode')
plt.plot([0,1.5],[0,1.5],'r--');
plt.ylim(-0.1,1.5)
plt.xlim(-0.1,1.5)
plt.ylabel("bpz photo-z")
plt.xlabel("true redshift")

Aqui já sairam os pits em comparação ao notebook 04 